<a href="https://colab.research.google.com/github/newmantic/BERT/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

In [2]:
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size, embed_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)

    def forward(self, x):
        return self.embedding(x)

In [3]:
class PositionalEncoding(nn.Module):
    def __init__(self, embed_size, max_len=512):
        super(PositionalEncoding, self).__init__()
        self.encoding = torch.zeros(max_len, embed_size)
        pos = torch.arange(0, max_len).unsqueeze(1).float()
        i = torch.arange(0, embed_size // 2).float()
        angle_rates = 1 / (10000 ** (2 * i / embed_size))
        self.encoding[:, 0::2] = torch.sin(pos * angle_rates)
        self.encoding[:, 1::2] = torch.cos(pos * angle_rates)
        self.encoding = self.encoding.unsqueeze(0)

    def forward(self, x):
        return x + self.encoding[:, :x.size(1), :]

In [4]:
class MultiHeadAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(MultiHeadAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert self.head_dim * heads == embed_size, "Embedding size must be divisible by heads"

        self.values = nn.Linear(self.head_dim, self.embed_size, bias=False)
        self.keys = nn.Linear(self.head_dim, self.embed_size, bias=False)
        self.queries = nn.Linear(self.head_dim, self.embed_size, bias=False)
        self.fc_out = nn.Linear(embed_size, embed_size)

    def forward(self, values, keys, query, mask):
        N = query.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        # Split embedding into self.heads pieces
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        queries = query.reshape(N, query_len, self.heads, self.head_dim)

        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])

        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))

        attention = torch.softmax(energy / (self.embed_size ** (1 / 2)), dim=3)

        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.embed_size
        )

        out = self.fc_out(out)
        return out

In [5]:
class FeedForward(nn.Module):
    def __init__(self, embed_size, ff_hidden_size, dropout):
        super(FeedForward, self).__init__()
        self.fc1 = nn.Linear(embed_size, ff_hidden_size)
        self.fc2 = nn.Linear(ff_hidden_size, embed_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.fc2(self.dropout(F.relu(self.fc1(x))))

In [6]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, ff_hidden_size, dropout):
        super(TransformerBlock, self).__init__()
        self.attention = MultiHeadAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)
        self.ff = FeedForward(embed_size, ff_hidden_size, dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        attention = self.attention(value, key, query, mask)
        x = self.dropout(self.norm1(attention + query))
        forward = self.ff(x)
        out = self.dropout(self.norm2(forward + x))
        return out

In [7]:
class BERT(nn.Module):
    def __init__(self, vocab_size, embed_size, num_layers, heads, ff_hidden_size, dropout, max_len, num_classes):
        super(BERT, self).__init__()
        self.token_embedding = TokenEmbedding(vocab_size, embed_size)
        self.position_encoding = PositionalEncoding(embed_size, max_len)
        self.layers = nn.ModuleList(
            [
                TransformerBlock(embed_size, heads, ff_hidden_size, dropout)
                for _ in range(num_layers)
            ]
        )
        self.fc_out = nn.Linear(embed_size, num_classes)

    def forward(self, x, mask):
        out = self.token_embedding(x)
        out = self.position_encoding(out)

        for layer in self.layers:
            out = layer(out, out, out, mask)

        out = self.fc_out(out[:, 0, :])
        return out

In [8]:
# Example Testing
vocab_size = 10000
embed_size = 128
num_layers = 2
heads = 8
ff_hidden_size = 512
dropout = 0.3
max_len = 512
num_classes = 2  # Binary classification

model = BERT(vocab_size, embed_size, num_layers, heads, ff_hidden_size, dropout, max_len, num_classes)

# Example input: batch size of 2, sequence length of 10
x = torch.randint(0, vocab_size, (2, 10))
mask = None  # No masking in this simplified example

# Forward pass
logits = model(x, mask)
print(logits)

tensor([[-0.4993, -1.7488],
        [ 1.5556, -0.1431]], grad_fn=<AddmmBackward0>)
